# Solution: The Silent Degradation

This notebook shows the correct solution to the monitoring drill.

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from typing import Dict, List

np.random.seed(42)

In [ ]:
def generate_data(n_samples, drift_factor=0, concept_drift=0):
    """Generate data with optional drift."""
    tenure = np.random.exponential(20, n_samples)
    spend = np.random.normal(100, 30, n_samples)
    tickets = np.random.poisson(2, n_samples)
    
    if drift_factor > 0:
        spend = spend * (1 + drift_factor * 0.5)
        tenure = tenure * (1 + drift_factor * 0.3)
    
    churn_prob = 0.2 + 0.2 * (tickets > 3) - 0.1 * (spend > 100)
    
    if concept_drift > 0:
        churn_prob = 0.2 - 0.2 * (spend > 100) + 0.05 * (tickets > 3)
    
    churn_prob = np.clip(churn_prob, 0.05, 0.95)
    churned = (np.random.random(n_samples) < churn_prob).astype(int)
    
    return pd.DataFrame({
        'tenure': tenure,
        'spend': spend,
        'tickets': tickets,
        'churned': churned
    })

# Generate training data and model
train_data = generate_data(2000)
features = ['tenure', 'spend', 'tickets']

model = GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=42)
model.fit(train_data[features], train_data['churned'])

baseline_auc = roc_auc_score(train_data['churned'], model.predict_proba(train_data[features])[:, 1])
print(f"Baseline AUC: {baseline_auc:.3f}")

## The Solution: Proper Monitoring with Alerts

In [ ]:
def calculate_psi(expected, actual, buckets=10):
    """Calculate Population Stability Index."""
    bins = np.percentile(expected, np.linspace(0, 100, buckets + 1))
    bins[0], bins[-1] = -np.inf, np.inf
    
    exp_counts = np.histogram(expected, bins)[0] / len(expected)
    act_counts = np.histogram(actual, bins)[0] / len(actual)
    
    exp_counts = np.clip(exp_counts, 0.0001, None)
    act_counts = np.clip(act_counts, 0.0001, None)
    
    return np.sum((act_counts - exp_counts) * np.log(act_counts / exp_counts))


class ModelMonitor:
    """Model monitoring with drift detection and alerts."""
    
    def __init__(self, baseline_data, baseline_auc, thresholds):
        self.baseline_data = baseline_data
        self.baseline_auc = baseline_auc
        self.thresholds = thresholds
        self.alerts = []
    
    def check(self, current_data, predictions, actuals=None):
        """Check for drift and performance degradation."""
        alerts = []
        
        # Check data drift (PSI)
        for feature in ['tenure', 'spend', 'tickets']:
            psi = calculate_psi(
                self.baseline_data[feature].values,
                current_data[feature].values
            )
            if psi > self.thresholds['psi_critical']:
                alerts.append(f"🚨 CRITICAL: {feature} PSI={psi:.2f}")
            elif psi > self.thresholds['psi_warning']:
                alerts.append(f"⚠️ WARNING: {feature} PSI={psi:.2f}")
        
        # Check performance (if we have ground truth)
        if actuals is not None:
            current_auc = roc_auc_score(actuals, predictions)
            auc_drop = self.baseline_auc - current_auc
            
            if auc_drop > self.thresholds['auc_critical']:
                alerts.append(f"🚨 CRITICAL: AUC dropped {auc_drop:.2%}")
            elif auc_drop > self.thresholds['auc_warning']:
                alerts.append(f"⚠️ WARNING: AUC dropped {auc_drop:.2%}")
        
        self.alerts.extend(alerts)
        return alerts


# Configure thresholds
thresholds = {
    'psi_warning': 0.1,
    'psi_critical': 0.25,
    'auc_warning': 0.03,
    'auc_critical': 0.05
}

monitor = ModelMonitor(train_data, baseline_auc, thresholds)
print("✓ ModelMonitor created")

## Simulating Production with Monitoring

In [ ]:
print("=== Production Monitoring Over 12 Weeks ===")
weekly_data = []

for week in range(12):
    drift = min(week / 8, 1.0) * 0.6
    concept_drift = max(0, (week - 4) / 8) * 0.8
    
    prod_data = generate_data(500, drift_factor=drift, concept_drift=concept_drift)
    preds = model.predict_proba(prod_data[features])[:, 1]
    actual_auc = roc_auc_score(prod_data['churned'], preds)
    
    alerts = monitor.check(prod_data, preds, prod_data['churned'])
    
    weekly_data.append({
        'week': week + 1,
        'auc': actual_auc,
        'alerts': len(alerts)
    })
    
    if alerts:
        print(f"\nWeek {week + 1} (AUC: {actual_auc:.3f}):")
        for alert in alerts:
            print(f"  {alert}")

print(f"\n✓ Monitoring caught {len(monitor.alerts)} alerts total!")

## Visualization

In [ ]:
df_weekly = pd.DataFrame(weekly_data)

plt.figure(figsize=(10, 5))
plt.plot(df_weekly['week'], df_weekly['auc'], 'b-o', linewidth=2, markersize=8)
plt.axhline(y=baseline_auc, color='g', linestyle='--', label='Baseline')
plt.axhline(y=baseline_auc - 0.03, color='orange', linestyle='--', label='Warning')
plt.axhline(y=baseline_auc - 0.05, color='r', linestyle='--', label='Critical')

# Mark alert weeks
for _, row in df_weekly.iterrows():
    if row['alerts'] > 0:
        plt.scatter(row['week'], row['auc'], color='red', s=200, zorder=5)

plt.xlabel('Week')
plt.ylabel('AUC')
plt.title('Model Performance with Monitoring (Red = Alerts Fired)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Verification
assert len(monitor.alerts) > 0, "Monitor should have caught alerts"
assert any('CRITICAL' in a for a in monitor.alerts), "Should have critical alerts"

print("\n✓ All checks passed!")
print(f"✓ Total alerts: {len(monitor.alerts)}")
print("✓ Degradation was detected before business impact!")

## Key Insight

Effective monitoring requires:

1. **Track multiple metrics** - PSI for data drift, accuracy/AUC for performance
2. **Set tiered thresholds** - Warning for investigation, Critical for action
3. **Alert early** - Before business metrics decline
4. **Automate decisions** - Clear triggers for retraining

| Metric | Warning | Critical |
|--------|---------|----------|
| PSI | > 0.1 | > 0.25 |
| AUC Drop | > 3% | > 5% |
| Pred Mean Shift | > 1 std | > 2 std |